In [7]:
import pymysql
import requests
import datetime
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import sys
import random
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
session = requests.Session()
session.keep_alive = True

custom_headers = {
	'Connection': 'Keep-Alive'
}

profile = webdriver.FirefoxProfile()
browser = webdriver.Firefox(profile)	

In [3]:
html = "http://www.sciencedirect.com/science/search"
browser.get(html)
html_source = browser.page_source
bs = BeautifulSoup(html_source,"lxml")

[]

In [13]:
el = browser.find_element_by_id('keywordOpt')
for option in el.find_elements_by_tag_name('option'):
    if option.text == 'Title':
        option.click()
        break

In [ ]:
link = bs.findNextSibling